In [1]:
pip install transformers datasets torch pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import pandas as pd
from transformers import EarlyStoppingCallback
from sklearn.model_selection import train_test_split

# Path ke file CSV dataset Anda
dataset_path = '/content/drive/MyDrive/Colab Notebooks/ID_Quora_Paraphrasing_train.csv'

# Membaca dataset menggunakan pandas
df = pd.read_csv(dataset_path)
df = df.sample(frac=0.2, random_state=42)
# Menampilkan beberapa contoh data
print("Contoh data:")
print(df.head())

# Membagi dataset menjadi train (80%) dan validation (20%)
train_df, validation_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Jumlah data train: {len(train_df)}")
print(f"Jumlah data validation: {len(validation_df)}")


Contoh data:
                                               question_1  \
45459      Apa cara yang berbeda untuk menghasilkan uang?   
119077  Apa alasan di balik larangan pemerintah pada 5...   
113777    Apa buku-buku terbaik dalam pembelajaran mesin?   
8999    Jika saya menjadi ahli saraf, apa yang harus s...   
58429      Apa cara terbaik untuk mengembangkan aplikasi?   

                                               question_2  
45459                         Apa cara menghasilkan uang?  
119077  Mengapa Modi Memo Rs 500 & Rs 1000 Catatan? Da...  
113777  Apa buku-buku terbaik tentang pembelajaran mesin?  
8999    Apa yang harus saya lakukan untuk menjadi ahli...  
58429              Bagaimana cara mengembangkan aplikasi?  
Jumlah data train: 21453
Jumlah data validation: 5364


In [3]:
from datasets import Dataset

# Mengonversi DataFrame pandas ke Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)

# Menampilkan beberapa contoh data
print("Contoh data train:")
print(train_dataset[0])
print("Contoh data validation:")
print(validation_dataset[0])


Contoh data train:
{'question_1': 'Akankah hidup menjadi lebih baik?', 'question_2': 'Apakah hidup selalu menjadi lebih baik?', '__index_level_0__': 47021}
Contoh data validation:
{'question_1': 'Kualitas apa yang harus Anda cari saat memutuskan apakah akan mengikuti seseorang di Quora?', 'question_2': 'Apa yang Anda pertimbangkan ketika Anda mengikuti seseorang di Quora?', '__index_level_0__': 90628}


In [4]:
from transformers import AutoTokenizer

# Nama model
model_name = "bigscience/bloom-560m"

# Memuat tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Menambahkan token pad jika diperlukan
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Fungsi tokenisasi untuk kolom 'original'
def tokenize_function(example):
    return tokenizer(
        example['question_1'],
        max_length=128,
        truncation=True,
        padding='max_length'
    )

# Fungsi untuk menyiapkan label dari kolom 'paraphrased'
def prepare_labels(example):
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example['question_2'],
            max_length=128,
            truncation=True,
            padding='max_length'
        )
    example['labels'] = labels['input_ids']
    return example

# Menerapkan tokenisasi dan menyiapkan label ke train dan validation dataset
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_train = tokenized_train.map(prepare_labels, batched=True)

tokenized_validation = validation_dataset.map(tokenize_function, batched=True)
tokenized_validation = tokenized_validation.map(prepare_labels, batched=True)

# Menghapus kolom yang tidak diperlukan dan mengatur format dataset
columns_to_remove = ['question_1', 'question_2']
tokenized_train = tokenized_train.remove_columns(columns_to_remove)
tokenized_validation = tokenized_validation.remove_columns(columns_to_remove)

tokenized_train.set_format('torch')
tokenized_validation.set_format('torch')

# Menampilkan contoh data yang sudah ditokenisasi
print("Contoh data tokenized train:")
print(tokenized_train[0])
print("Contoh data tokenized validation:")
print(tokenized_validation[0])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Map:   0%|          | 0/21453 [00:00<?, ? examples/s]

Map:   0%|          | 0/21453 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/5364 [00:00<?, ? examples/s]

Map:   0%|          | 0/5364 [00:00<?, ? examples/s]

Contoh data tokenized train:
{'__index_level_0__': tensor(47021), 'input_ids': tensor([    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,

In [5]:
from transformers import AutoModelForCausalLM

# Memuat model
model = AutoModelForCausalLM.from_pretrained(model_name)

# Jika Anda menambahkan token khusus, perlu resize token embeddings
model.resize_token_embeddings(len(tokenizer))


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Embedding(250680, 1024)

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./bloom_paraphrase',          # Direktori output untuk model yang telah dilatih
    eval_strategy="epoch",                    # Strategi evaluasi diubah dari 'evaluation_strategy' ke 'eval_strategy'
    save_strategy="epoch",                    # Strategi penyimpanan disesuaikan agar sama dengan eval_strategy
    learning_rate=5e-5,                       # Learning rate
    per_device_train_batch_size=4,            # Ukuran batch untuk pelatihan
    per_device_eval_batch_size=4,             # Ukuran batch untuk validasi
    num_train_epochs=15,                      # Jumlah epoch maksimum
    weight_decay=0.01,                        # Weight decay
    save_total_limit=2,                       # Maksimum model yang disimpan
    logging_dir='./logs',                     # Direktori logging
    logging_steps=10,                         # Logging setiap 10 langkah
    load_best_model_at_end=True,              # Memuat model terbaik saat pelatihan selesai
    metric_for_best_model='eval_loss',         # Metrik yang dipantau untuk model terbaik
)


In [7]:
from transformers import BloomForCausalLM, BloomTokenizerFast

model_name = "bigscience/bloom-560m"
tokenizer = BloomTokenizerFast.from_pretrained(model_name)
model = BloomForCausalLM.from_pretrained(model_name)


In [21]:
def paraphrase_bloom_oneshot(text, true_paraphrase):
    # Membuat prompt untuk memastikan hanya parafrase yang diberikan
    prompt = f"Susun ulang kata-kata kalimat ini: {text}"

    # Tokenisasi input
    inputs = tokenizer(prompt, return_tensors="pt")

    # Melakukan prediksi menggunakan model dengan pengaturan yang lebih ketat
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=50,  # Adjust if longer summaries are needed
        num_beams=5,
        temperature=0.7,
        do_sample=True,
        no_repeat_ngram_size=3,
        early_stopping=True,
    )

    # Mendekode hasil output dan memproses hasil parafrase
    paraphrased = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Menyaring hasil parafrase dari output
    paraphrased_text = paraphrased.strip()

    # Menyusun hasil dalam format yang diinginkan
    result = {
        'original_text': text,
        'predicted_paraphrase': paraphrased_text,
        'true_paraphrase': true_paraphrase
    }

    return result

# Contoh penggunaan
# Mengambil data dari dataset
# original_text = validation_dataset[0]['question_1']
# true_paraphrase = validation_dataset[0]['question_2']

# result = paraphrase_bloom_oneshot(original_text, true_paraphrase)

# # Menampilkan hasil
# print(f"Original Text: {result['original_text']}")
# print(f"Predicted Paraphrase: {result['predicted_paraphrase']}")
# print(f"True Paraphrase: {result['true_paraphrase']}")
for i in range(5):  # Iterasi untuk 5 entri pertama
    original_text = validation_dataset['question_1'][i]  # Ambil teks asli
    true_paraphrase = validation_dataset['question_2'][i]  # Ambil true paraphrase

    # Memproses menggunakan fungsi paraphrase_bloom_oneshot
    result = paraphrase_bloom_oneshot(original_text, true_paraphrase)

    # Menampilkan hasil
    print(f"Original Text: {result['original_text']}")
    print(f"Predicted Paraphrase: {result['predicted_paraphrase']}")
    print(f"True Paraphrase: {result['true_paraphrase']}")
    print("=" * 50)



Original Text: Kualitas apa yang harus Anda cari saat memutuskan apakah akan mengikuti seseorang di Quora?
Predicted Paraphrase: Susun ulang kata-kata kalimat ini: Kualitas apa yang harus Anda cari saat memutuskan apakah akan mengikuti seseorang di Quora? Atau apakah Anda akan mencari seseorang di Facebook? Atau, apakah Anda ingin menemukan seseorang di Twitter? Atau Anda ingin mencari orang di LinkedIn? Atau bagaimana Anda akan menemukan orang yang Anda sukai di Instagram?
True Paraphrase: Apa yang Anda pertimbangkan ketika Anda mengikuti seseorang di Quora?
Original Text: Mengapa orang terkaya di Inggris kebanyakan imigran?
Predicted Paraphrase: Susun ulang kata-kata kalimat ini: Mengapa orang terkaya di Inggris kebanyakan imigran? Mengapa orang miskin di Inggris banyak orang yang tinggal di luar negeri? Kenapa orang kaya di Inggris mayoritas orang yang bekerja di negara lain? Mengapa orang-orang miskin di Eropa kebanyakan orang yang tidak memiliki pekerjaan? Kenapa orang-orang kaya 

In [10]:
pip install nltk


In [24]:
from transformers import AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu

# Pastikan sudah mendownload tokenizer sesuai model
tokenizer = AutoTokenizer.from_pretrained(model_name)

def calculate_bleu_score(predicted_paraphrase, true_paraphrase):
    # Tokenisasi kalimat prediksi dan true paraphrase menggunakan tokenizer dari transformers
    predicted_tokens = tokenizer.tokenize(predicted_paraphrase.lower())
    true_tokens = tokenizer.tokenize(true_paraphrase.lower())

    # Menghitung BLEU score
    bleu_score = sentence_bleu([true_tokens], predicted_tokens)

    return bleu_score

# Contoh penggunaan
# Iterasi pada range tertentu
for i in range(5):  # Mengambil 5 entri pertama
    original_text = validation_dataset['question_1'][i]  # Teks original
    true_paraphrase = validation_dataset['question_2'][i]  # Parafrase yang benar

    # Dapatkan hasil prediksi parafrase dari model
    predicted_paraphrase = paraphrase_bloom_oneshot(original_text, true_paraphrase)['predicted_paraphrase']

    # Hitung BLEU score
    bleu = calculate_bleu_score(predicted_paraphrase, true_paraphrase)

    # Menampilkan hasil
    print(f"Data ke-{i+1}")
    print(f"Original Text: {original_text}")
    print(f"Predicted Paraphrase: {predicted_paraphrase}")
    print(f"True Paraphrase: {true_paraphrase}")
    print(f"BLEU Score: {bleu:.4f}")
    print("=" * 50)


Data ke-1
Original Text: Kualitas apa yang harus Anda cari saat memutuskan apakah akan mengikuti seseorang di Quora?
Predicted Paraphrase: Susun ulang kata-kata kalimat ini: Kualitas apa yang harus Anda cari saat memutuskan apakah akan mengikuti seseorang di Quora? Ini adalah pertanyaan yang sering diajukan oleh para pengguna Quora. Ada banyak jawaban yang dapat Anda pilih untuk menjawab pertanyaan ini. Berikut adalah beberapa jawaban yang mungkin akan membantu Anda dalam menentukan apakah Anda akan menggunakan Quora atau tidak. Jika Anda memilih untuk menggunakan Qu
True Paraphrase: Apa yang Anda pertimbangkan ketika Anda mengikuti seseorang di Quora?
BLEU Score: 0.0684
Data ke-2
Original Text: Mengapa orang terkaya di Inggris kebanyakan imigran?
Predicted Paraphrase: Susun ulang kata-kata kalimat ini: Mengapa orang terkaya di Inggris kebanyakan imigran? Mengapa orang miskin di Inggris banyak orang yang bekerja di luar negeri? Kenapa orang kaya di Inggris jarang bekerja di negeri mere

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [23]:
def paraphrase_bloom_fewshot(text, true_paraphrase):
    # Membuat prompt untuk memastikan hanya parafrase yang diberikan
    prompt = f"Apa cara yang berbeda untuk menghasilkan uang?:  Apa cara menghasilkan uang?\n\nApa buku-buku terbaik dalam pembelajaran mesin?: Apa buku-buku terbaik tentang pembelajaran mesin?\n\n{text}:"

    # Tokenisasi input
    inputs = tokenizer(prompt, return_tensors="pt")

    # Melakukan prediksi menggunakan model dengan pengaturan yang lebih ketat
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=50,  # Adjust if longer summaries are needed
        num_beams=5,
        temperature=0.7,
        do_sample=True,
        no_repeat_ngram_size=3,
        early_stopping=True,
    )

    # Mendekode hasil output dan memproses hasil parafrase
    paraphrased = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Menyaring hasil parafrase dari output
    paraphrased_text = paraphrased.strip()

    # Menyusun hasil dalam format yang diinginkan
    result = {
        'original_text': text,
        'predicted_paraphrase': paraphrased_text,
        'true_paraphrase': true_paraphrase
    }

    return result

# Contoh penggunaan
# Mengambil data dari dataset
for i in range(5):  # Iterasi untuk 5 entri pertama
    original_text = validation_dataset['question_1'][i]  # Ambil teks asli
    true_paraphrase = validation_dataset['question_2'][i]  # Ambil true paraphrase

    # Memproses menggunakan fungsi paraphrase_bloom_oneshot
    result = paraphrase_bloom_fewshot(original_text, true_paraphrase)

    # Menampilkan hasil
    print(f"Original Text: {result['original_text']}")
    print(f"Predicted Paraphrase: {result['predicted_paraphrase']}")
    print(f"True Paraphrase: {result['true_paraphrase']}")
    print("=" * 50)



Original Text: Kualitas apa yang harus Anda cari saat memutuskan apakah akan mengikuti seseorang di Quora?
Predicted Paraphrase: Apa cara yang berbeda untuk menghasilkan uang?:  Apa cara menghasilkan uang?

Apa buku-buku terbaik dalam pembelajaran mesin?: Apa buku-buku terbaik tentang pembelajaran mesin?

Kualitas apa yang harus Anda cari saat memutuskan apakah akan mengikuti seseorang di Quora?: Kualitas mana yang perlu dipertimbangkan saat memilih seseorang untuk bergabung dengan Quora?

Apa yang bisa Anda lakukan untuk mendapatkan uang dengan cepat?: Bagaimana cara mendapatkan uang cepat?
True Paraphrase: Apa yang Anda pertimbangkan ketika Anda mengikuti seseorang di Quora?
Original Text: Mengapa orang terkaya di Inggris kebanyakan imigran?
Predicted Paraphrase: Apa cara yang berbeda untuk menghasilkan uang?:  Apa cara menghasilkan uang?

Apa buku-buku terbaik dalam pembelajaran mesin?: Apa buku-buku terbaik tentang pembelajaran mesin?

Mengapa orang terkaya di Inggris kebanyakan im

In [25]:
from transformers import AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu

# Pastikan sudah mendownload tokenizer sesuai model
tokenizer = AutoTokenizer.from_pretrained(model_name)

def calculate_bleu_score(predicted_paraphrase, true_paraphrase):
    # Tokenisasi kalimat prediksi dan true paraphrase menggunakan tokenizer dari transformers
    predicted_tokens = tokenizer.tokenize(predicted_paraphrase.lower())
    true_tokens = tokenizer.tokenize(true_paraphrase.lower())

    # Menghitung BLEU score
    bleu_score = sentence_bleu([true_tokens], predicted_tokens)

    return bleu_score

# Contoh penggunaan
# Iterasi pada range tertentu
for i in range(5):  # Mengambil 5 entri pertama
    original_text = validation_dataset['question_1'][i]  # Teks original
    true_paraphrase = validation_dataset['question_2'][i]  # Parafrase yang benar

    # Dapatkan hasil prediksi parafrase dari model
    predicted_paraphrase = paraphrase_bloom_fewshot(original_text, true_paraphrase)['predicted_paraphrase']

    # Hitung BLEU score
    bleu = calculate_bleu_score(predicted_paraphrase, true_paraphrase)

    # Menampilkan hasil
    print(f"Data ke-{i+1}")
    print(f"Original Text: {original_text}")
    print(f"Predicted Paraphrase: {predicted_paraphrase}")
    print(f"True Paraphrase: {true_paraphrase}")
    print(f"BLEU Score: {bleu:.4f}")
    print("=" * 50)


Data ke-1
Original Text: Kualitas apa yang harus Anda cari saat memutuskan apakah akan mengikuti seseorang di Quora?
Predicted Paraphrase: Apa cara yang berbeda untuk menghasilkan uang?:  Apa cara menghasilkan uang?

Apa buku-buku terbaik dalam pembelajaran mesin?: Apa buku-buku terbaik tentang pembelajaran mesin?

Kualitas apa yang harus Anda cari saat memutuskan apakah akan mengikuti seseorang di Quora?: Kualitas yang perlu dipertimbangkan saat memilih seseorang untuk bergabung dengan Quora.

Apa yang harus dipertimbangkan ketika memilih situs web untuk belajar bahasa Inggris?: Apakah situs web yang akan Anda pilih memiliki kualitas yang baik atau tidak?
True Paraphrase: Apa yang Anda pertimbangkan ketika Anda mengikuti seseorang di Quora?
BLEU Score: 0.0593
Data ke-2
Original Text: Mengapa orang terkaya di Inggris kebanyakan imigran?
Predicted Paraphrase: Apa cara yang berbeda untuk menghasilkan uang?:  Apa cara menghasilkan uang?

Apa buku-buku terbaik dalam pembelajaran mesin?: Ap